In [4]:
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps xformers peft accelerate bitsandbytes
!pip install transformers datasets
!pip install langchain
!pip install langchain_community

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-xz8plu5f/unsloth_bd10ca6c71e145f2abc04f487bc468f0
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-xz8plu5f/unsloth_bd10ca6c71e145f2abc04f487bc468f0
  Resolved https://github.com/unslothai/unsloth.git to commit f3f9d148874389996636e4fd110c7255b9f62c77
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [3]:
from unsloth import FastLanguageModel, is_bfloat16_supported
import torch
import json
from datasets import load_dataset
from trl import SFTTrainer
from transformers import TrainingArguments
import pandas as pd
from langchain_core.prompts import PromptTemplate # Corrected import path

from langchain_community.llms import Ollama
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.chains import LLMChain
from langchain_community.document_loaders import TextLoader
import os

DATA_PATH = "train.csv"
OUTPUT_PATH_DATASET = "/content/formatted_news_dataset_chat_data_complement.json"
max_seq_length = 2048
dtype = None
load_in_4bit = True
fourbit_models = [
    "unsloth/mistral-7b-v0.3-bnb-4bit",
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/llama-3-8b-bnb-4bit",
    "unsloth/llama-3-8b-Instruct-bnb-4bit",
    "unsloth/llama-3-70b-bnb-4bit",
    "unsloth/Phi-3-mini-4k-instruct",
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/gemma-7b-bnb-4bit",
]


ModuleNotFoundError: No module named 'langchain.callbacks'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [19]:
def format_dataset_into_model_input(data):
    def separate_text(dialog):
        input_text = dialog[0]
        response = dialog[1]
        instruction = "Você é uma psicóloga clínica formada, ética e empática. Retorne conselhos bons para as questões apresentadas. com educação e profissionalismo, sempre sugira que o paciente procure um médico humano e especialista"
        return instruction, input_text, response

    # Inicializando as listas para armazenar os dados
    instructions = []
    inputs = []
    outputs = []

    # Processando o dataset
    for dialog in data:
        instruction, input_text, response = separate_text(dialog)
        instructions.append(instruction)
        inputs.append(input_text)
        outputs.append(response)

    # Criando o dicionário final
    formatted_data = {
        "instruction": instructions,
        "input": inputs,
        "output": outputs
    }

    # Salvando o resultado em um arquivo JSON
    with open(OUTPUT_PATH_DATASET, 'w') as output_file:
        json.dump(formatted_data, output_file, indent=4)

    print(f"Dataset salvo em {OUTPUT_PATH_DATASET}")

In [20]:
data = load_dataset("csv", data_files=DATA_PATH)

ds = load_dataset("Ayansk11/Mental_health_data_conversational")

df_train = data['train'].to_pandas()
df_complement = ds['train'].to_pandas()
print(df_complement.head())
# display(df_train.tail())

# To iterate over the dataset, access the 'train' split
# and then iterate over its elements. Each element is a dictionary.
# Here's how to print the first element:
dataList = df_complement.values.tolist();
# for dialog in dataList:
#     print(dialog[1])
format_dataset_into_model_input(dataList)

# model, tokenizer = FastLanguageModel.from_pretrained(
#     model_name = "unsloth/llama-3-8b-bnb-4bit",
#     max_seq_length = max_seq_length,
#     dtype = dtype,
#     load_in_4bit = load_in_4bit,
# )

                                             Context  \
0  I'm going through some things with my feelings...   
1  I'm going through some things with my feelings...   
2  I'm going through some things with my feelings...   
3  I'm going through some things with my feelings...   
4  I'm going through some things with my feelings...   

                                            Response  
0  If everyone thinks you're worthless, then mayb...  
1  Hello, and thank you for your question and see...  
2  First thing I'd suggest is getting the sleep y...  
3  Therapy is essential for those that are feelin...  
4  I first want to let you know that you are not ...  
Dataset salvo em /content/formatted_news_dataset_chat_data_complement.json


In [4]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

==((====))==  Unsloth 2026.1.2: Fast Llama patching. Transformers: 4.57.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/198 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

In [21]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):

        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

from datasets import load_dataset

OUTPUT_PATH_DATASET = "/content/formatted_news_dataset_chat_data_complement.json"

dataset = load_dataset("json", data_files=OUTPUT_PATH_DATASET, split = "train")
dataset = dataset.map(formatting_prompts_func, batched = True,)

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/3512 [00:00<?, ? examples/s]

In [22]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = [
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
    ],
    lora_alpha = 16,
    lora_dropout = 0.0,
    bias = "none",
    use_gradient_checkpointing = True,
    random_state = 3407,
)


trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 1,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

Unsloth: Already have LoRA adapters! We shall skip this step.


Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/3512 [00:00<?, ? examples/s]

In [23]:
trainer_stats = trainer.train()

The model is already on multiple devices. Skipping the move to device specified in `args`.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 3,512 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 41,943,040 of 8,072,204,288 (0.52% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,1.818000
2,1.909300
3,1.902100
4,1.828100
5,1.722100
6,1.417300
7,1.645800
8,1.766200
9,1.826800
10,1.695300


wandb: WARNING URL not available in offline run


train/epoch,▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▆▇▇▇▇█████
train/global_step,▁▁▁▁▁▂▂▂▂▃▃▃▃▃▃▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇▇███
train/grad_norm,█▃▄▂▅▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▂▁▁▁▁▂▂▂▁▁▂▁▁▂▁▂▂▁▂
train/learning_rate,▁▂▄▅▇████▇▇▇▇▇▆▆▆▆▆▆▅▅▅▅▅▄▄▄▄▄▃▃▃▃▂▂▂▂▁▁
train/loss,▇██▇▆▇▁▂▅▅▆▆▂▅▅▃▅▃▂▂▄▅▄▅▆▄▄▆▇▂▄▃▄▃▄▆▅▂▇▇
total_flos,9541021848748032.0
train/epoch,0.13667
train/global_step,60
train/grad_norm,0.44631
train/learning_rate,0.0
train/loss,1.7995


In [24]:

FastLanguageModel.for_inference(model)
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Voc\u00ea \u00e9 uma psic\u00f3loga cl\u00ednica formada, \u00e9tica e emp\u00e1tica. Retorne conselhos bons para as quest\u00f5es apresentadas. com educa\u00e7\u00e3o e profissionalismo, sempre sugira que o paciente procure um médico humano e especialista.",
        "I would like to improve my self-control, because there are times when I can't manage my emotions and end up having exaggerated reactions.", # input
        "",
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)

["<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nVocê é uma psicóloga clínica formada, ética e empática. Retorne conselhos bons para as questões apresentadas. com educação e profissionalismo, sempre sugira que o paciente procure um médico humano e especialista.\n\n### Input:\nI would like to improve my self-control, because there are times when I can't manage my emotions and end up having exaggerated reactions.\n\n### Response:\nI would suggest that you first make an attempt to understand what is happening when you have these reactions. \xa0For example, what are you feeling, what are you thinking, and what are you wanting? \xa0Once you have a sense of what is going on, you can then start to think of ways to change"]

In [9]:

FastLanguageModel.for_inference(model)
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Voc\u00ea \u00e9 uma psic\u00f3loga cl\u00ednica formada, \u00e9tica e emp\u00e1tica. N\u00e3o realize diagn\u00f3sticos. N\u00e3o substitua atendimento profissional.",
        "I would like to improve my self-control, because there are times when I can't manage my emotions and end up having exaggerated reactions.", # input
        "",
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Você é uma psicóloga clínica formada, ética e empática. Não realize diagnósticos. Não substitua atendimento profissional.

### Input:
I would like to improve my self-control, because there are times when I can't manage my emotions and end up having exaggerated reactions.

### Response:
Não realize diagnósticos. Não substitua atendimento profissional.

Não realize diagnósticos. Não substitua atendimento profissional.

Não realize diagnósticos. Não substitua atendimento profissional.

Não realize diagnósticos. Não substitua atendimento profissional.

Não realize diagnósticos. Não substitua atendimento profissional.

Não realize diagnósticos. Não substitua atendimento profissional.

Não realize diagnósticos. Não substitua atendimento profissional.

Não realize diagnósticos. Não substitua atendimento

In [25]:
from langchain.prompts import PromptTemplate

# model.invoke

# template = "Descreva as principais características de um produto chamado {produto}."

# llm = OpenAI(temperature=0.7)

# prompt = PromptTemplate(template=template, input_variables=["produto"])

# prompt_completo = prompt.format(produto="Smartphone X")

# resposta = llm.invoke(prompt_completo)
# print(resposta)

ModuleNotFoundError: No module named 'langchain.prompts'